<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: Notebook for Peer Assignment</font></h1>


# Introduction

Using this Python notebook you will:

1.  Understand three Chicago datasets
2.  Load the three datasets into three tables in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Understand the datasets

To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:

1.  Socioeconomic Indicators in Chicago
2.  Chicago Public Schools
3. Chicago Crime Data

### 1. Socioeconomic Indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.


### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

### 3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.


### Store the datasets in database tables

To analyze the data using SQL, it first needs to be loaded into SQLite DB.
We will create three tables in as under:

1.  **CENSUS_DATA**
2.  **CHICAGO_PUBLIC_SCHOOLS**
3.  **CHICAGO_CRIME_DATA**

Let us now load the ipython-sql  extension and establish a connection with the database

*   Here you will be loading the csv files into the pandas Dataframe and then loading the data into the above mentioned sqlite tables.

*   Next you will be connecting to the sqlite database  **FinalDB**.



In [1]:
import pandas as pd
import sqlite3  # To manage SQLite db

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///FinalDB.db

In [4]:
CENSUS_DATA = pd.read_csv('/Users/salahkaf/Downloads/ChicagoCensusData.csv')
CHICAGO_PUBLIC_SCHOOLS = pd.read_csv('/Users/salahkaf/Downloads/ChicagoPublicSchools.csv')
CHICAGO_CRIME_DATA = pd.read_csv('/Users/salahkaf/Downloads/ChicagoCrimeData.csv')

In [5]:
con = sqlite3.connect("FinalDB.db") #

In [6]:
CENSUS_DATA.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")
CHICAGO_PUBLIC_SCHOOLS.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False,method="multi")
CHICAGO_CRIME_DATA.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False,method="multi")

/Users/salahkaf/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [7]:
%sql SELECT name FROM sqlite_master WHERE type='table';


 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_PUBLIC_SCHOOLS
CHICAGO_CRIME_DATA


## Problems

Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.


In [8]:
%%sql 
SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


### Problem 2

##### List community areas with per capita income less than 11000.


In [9]:
%sql select community_area_number as "Area Number", \
community_area_name as "Area Name", \
per_capita_income as "Per Capita Income" \
from census_data \
where per_capita_income < 11000

 * sqlite:///FinalDB.db
Done.


Area Number,Area Name,Per Capita Income
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [10]:
%%sql
SELECT CASE_NUMBER, DATE FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%MINOR'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DATE
HL266884,2005-03-31
HK238408,2004-03-13


### Problem 4

##### List all kidnapping crimes involving a child?


In [11]:
%sql SELECT CASE_NUMBER from CHICAGO_CRIME_DATA \
WHERE PRIMARY_TYPE = 'KIDNAPPING';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HN144152


### Problem 5

##### What kinds of crimes were recorded at schools?


In [12]:
%sql SELECT DISTINCT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA \
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6

##### List the average safety score for each type of school.


In [13]:
%sql SELECT "Elementary, Middle, or High School" AS SCHOOL_TYPE, AVG(SAFETY_SCORE) AS AVG FROM CHICAGO_PUBLIC_SCHOOLS \
GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


SCHOOL_TYPE,AVG
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [14]:
%sql select Community_area_number, \
Community_area_name, \
percent_households_below_poverty \
from census_data \
order by percent_households_below_poverty desc limit 5 

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone?


In [15]:
TABLE = %sql select community_area_number as "Most Frequent Community Area Number", \
COUNT(community_area_number) as "Number of crimes" \
from CHICAGO_CRIME_DATA \
group by community_area_number \
order by count(community_area_number) DESC limit 1; 
print (TABLE)
print ("Most frequent area number is 25, with 43 committed crimes")
%sql select community_area_name from census_data \
where community_area_number = 25

 * sqlite:///FinalDB.db
Done.
+-------------------------------------+------------------+
| Most Frequent Community Area Number | Number of crimes |
+-------------------------------------+------------------+
|                 25.0                |        43        |
+-------------------------------------+------------------+
Most frequent area number is 25, with 43 committed crimes
 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin


Double-click **here** for a hint

<!--
Query for the 'community area number' that is most crime prone.
-->


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [16]:
%%sql
SELECT COMMUNITY_AREA_NAME from CENSUS_DATA
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [17]:
%%sql 
select community_area_name
as "Community Name with Most Number of Crimes "
from census_data 
where  Community_area_number
in (select community_area_number from
(select community_area_number, COUNT(community_area_number)
from CHICAGO_CRIME_DATA 
group by community_area_number 
order by count(community_area_number) DESC limit 1))

 * sqlite:///FinalDB.db
Done.


Community Name with Most Number of Crimes
Austin


Copyright © 2020 This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).


## Author(s)

<h4> Hima Vasudevan </h4>
<h4> Rav Ahuja </h4>
<h4> Ramesh Sannreddy </h4>

## Contribtuor(s)

<h4> Malika Singla </h4>

## Change log

| Date       | Version | Changed by        | Change Description                             |
| ---------- | ------- | ----------------- | ---------------------------------------------- |
| 2022-03-04 | 2.5     | Lakshmi Holla     | Changed markdown.                              |
| 2021-05-19 | 2.4     | Lakshmi Holla     | Updated the question                           |
| 2021-04-30 | 2.3     | Malika Singla     | Updated the libraries                          |
| 2021-01-15 | 2.2     | Rav Ahuja         | Removed problem 11 and fixed changelog         |
| 2020-11-25 | 2.1     | Ramesh Sannareddy | Updated the problem statements, and datasets   |
| 2020-09-05 | 2.0     | Malika Singla     | Moved lab to course repo in GitLab             |
| 2018-07-18 | 1.0     | Rav Ahuja         | Several updates including loading instructions |
| 2018-05-04 | 0.1     | Hima Vasudevan    | Created initial version                        |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
